# Bước 2b: “Làm trắng” (Whitening) toàn bộ Database Train (v2 - inner_content)


## Mục tiêu của script
Script này dùng để:
- **Tải database vector gốc (train)** đã gộp sẵn: `combined_train_db_vectors_v2_inner_content.pt`
- **Tải dữ liệu Mahalanobis** (mean + whitening matrix theo từng layer) từ Bước 2a: `mahalanobis_data_v2_inner_content.pt`
- Thực hiện **whitening** cho toàn bộ vector trong database:
  - Với từng layer `layer_1 → layer_12`
  - Xử lý theo **batch** để tiết kiệm RAM/VRAM
- Lưu ra file database mới đã whiten:
  - `combined_train_db_vectors_v2_WHITENED.pt`


## Thư viện sử dụng
- `torch`: load/save tensor, tính toán GPU, matmul, no_grad.
- `os`: thao tác đường dẫn file/thư mục.
- `tqdm.auto`: progress bar cho vòng lặp theo layer.


## Cấu hình tham số
- `NUM_LAYERS = 12`  
  Số layer cần xử lý (layer_1 → layer_12).

- `D_FEATURES = 2304`  
  Kích thước vector embedding.

- `BATCH_SIZE = 512`  
  Xử lý theo batch để:
  - giảm peak memory khi đưa dữ liệu lên GPU
  - tránh tràn VRAM khi `N_samples` lớn

- `device = cuda/cpu`  
  Nếu có GPU → chạy trên CUDA để whitening nhanh hơn.

## Thiết lập đường dẫn I/O

### Input 1 — Database gốc (chưa whiten)
- `combined_train_db_vectors_v2_inner_content.pt`  
  Chứa tensor `db_vectors` shape:
  - `(N_samples, NUM_LAYERS, D_FEATURES)`

### Input 2 — Dữ liệu Mahalanobis (từ Bước 2a)
- `mahalanobis_data_v2_inner_content.pt`  
  Chứa:
  - `means[layer_k]` shape `(2304,)`
  - `whitening_matrices[layer_k]` shape `(2304, 2304)`

### Output — Database đã whiten
- `combined_train_db_vectors_v2_WHITENED.pt`  
  Lưu tensor `whitened_db_vectors` shape giống input:
  - `(N_samples, NUM_LAYERS, D_FEATURES)`

## Luồng thực thi chính (`__main__`)

## 1) Tải database gốc
- Load:
  - `db_vectors = torch.load(db_vectors_file)`
- Lấy shape:
  - `(N_samples, n_layers, D_features_loaded)`
- Kiểm tra đúng cấu hình:
  - `n_layers == NUM_LAYERS`
  - `D_features_loaded == D_FEATURES`

Nếu load lỗi → dừng chương trình.

## 2) Tải dữ liệu Mahalanobis và chuyển lên GPU
- Load:
  - `mahalanobis_data = torch.load(mahalanobis_data_file)`
- Với từng layer:
  - đưa `mean` lên GPU:
    - `means[layer_name].to(device)`
  - đưa `whitening_matrix` lên GPU:
    - `whitening_matrices[layer_name].to(device)`

Mục đích:
- Khi whitening theo batch, phép trừ mean và matmul chạy trên GPU.

## 3) Chuẩn bị tensor output để lưu kết quả
Tạo tensor mới để chứa database đã whiten:
- `whitened_db_vectors = torch.empty_like(db_vectors, device='cpu', dtype=torch.float32)`

Điểm quan trọng:
- Đặt **trên CPU** để:
  - tiết kiệm VRAM
  - chỉ đưa từng batch lên GPU rồi đưa kết quả về CPU

## 4) Whitening theo từng layer và theo batch

### Tắt gradient để tiết kiệm bộ nhớ và tăng tốc
- Bọc toàn bộ bằng:
  - `with torch.no_grad():`
Vì đây là bước transform dữ liệu, không cần backprop.


### Vòng lặp theo layer
Với mỗi `i` trong `range(NUM_LAYERS)`:
- Đặt:
  - `layer_name = f"layer_{i+1}"`
  - `tensor_index = i`

Lấy dữ liệu của layer hiện tại:
- `mean = mahalanobis_data['means'][layer_name]` → shape `(2304,)`
- `W = mahalanobis_data['whitening_matrices'][layer_name]` → shape `(2304, 2304)

### Vòng lặp theo batch
Với `start_idx` chạy từ `0` đến `N_samples` bước `BATCH_SIZE`:
- `end_idx = min(start_idx + BATCH_SIZE, N_samples)`

#### 4.1 Trích batch dữ liệu và đưa lên GPU
- `X_batch = db_vectors[start_idx:end_idx, tensor_index, :].to(device)`
- Shape:
  - `(B, 2304)` với `B = end_idx - start_idx`

#### 4.2 Center dữ liệu (trừ mean)
- `X_centered = X_batch - mean`

Ghi chú:
- `mean` có shape `(2304,)` sẽ được **broadcast** tự động để trừ cho từng dòng `(B, 2304)`.


#### 4.3 Whitening (nhân với ma trận làm trắng)
- `X_whitened = torch.matmul(X_centered, W.T)`

Giải thích shape:
- `X_centered`: `(B, 2304)`
- `W.T`: `(2304, 2304)`
- Kết quả:
  - `(B, 2304)`

Tại sao dùng `W.T`?
- Do convention: vector là row-vector `(1 x D)`
- Whitening: `x_whiten = (x - mean) @ W^T` (để đúng chiều nhân)


#### 4.4 Lưu kết quả về CPU
- `whitened_db_vectors[start_idx:end_idx, tensor_index, :] = X_whitened.cpu()`

Mục đích:
- không giữ toàn bộ output trên GPU
- chỉ compute trên GPU theo batch, rồi lưu về CPU

## 5) Giải phóng bộ nhớ sau mỗi layer
Sau khi xử lý xong 1 layer:
- Xoá `mean` và `W` khỏi dict để giải phóng VRAM:
  - `del mahalanobis_data['means'][layer_name]`
  - `del mahalanobis_data['whitening_matrices'][layer_name]`
- Dọn VRAM:
  - `torch.cuda.empty_cache()`

Điểm lợi:
- Nếu ma trận `(2304 x 2304)` khá lớn, giữ 12 layer cùng lúc sẽ tốn nhiều VRAM.

## 6) Lưu database đã whiten
- `torch.save(whitened_db_vectors, output_db_file)`

Output là tensor float32 trên CPU, shape:
- `(N_samples, 12, 2304)`

## Kết quả đầu ra
Sau khi chạy xong, bạn thu được file:
- `combined_train_db_vectors_v2_WHITENED.pt`

File này dùng cho các bước sau như:
- xây dựng retrieval / nearest neighbor trên embedding đã whiten
- tính distance ổn định hơn (giảm tương quan giữa các chiều)
- chuẩn hoá theo từng layer để so sánh nhất quán

In [ ]:
# Import các thư viện cần thiết
import torch
import os
from tqdm.auto import tqdm

print("--- Bắt đầu Bước 2b: 'Làm trắng' (Whitening) toàn bộ Database Train ---")

# --- 1. CẤU HÌNH ---
NUM_LAYERS = 12
D_FEATURES = 2304
BATCH_SIZE = 512 # Xử lý theo batch để tiết kiệm RAM/VRAM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# --- 2. THIẾT LẬP CÁC ĐƯỜNG DẪN ---
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'
db_cache_dir = os.path.join(drive_base_path, 'cached_databases')

# --- INPUT 1: Database GỐC (v2 - inner content) ---
db_vectors_file = os.path.join(db_cache_dir, 'combined_train_db_vectors_v2_inner_content.pt')

# --- INPUT 2: File chứa các ma trận Mahalanobis (từ Bước 2a) ---
mahalanobis_data_file = os.path.join(db_cache_dir, 'mahalanobis_data_v2_inner_content.pt')

# --- OUTPUT: File database đã "làm trắng" (whitened) ---
output_db_file = os.path.join(db_cache_dir, 'combined_train_db_vectors_v2_WHITENED.pt') # Tên file mới

print(f"File database input: {db_vectors_file}")
print(f"File ma trận input: {mahalanobis_data_file}")
print(f"File database output: {output_db_file}")

# --- 3. THỰC THI CHƯƠNG TRÌNH ---
if __name__ == "__main__":

    # --- Tải Database gốc ---
    try:
        print(f"\nĐang tải database gốc: {db_vectors_file}...")
        db_vectors = torch.load(db_vectors_file)
        N_samples, n_layers, D_features_loaded = db_vectors.shape
        print(f" -> Tải thành công! Shape: ({N_samples}, {n_layers}, {D_features_loaded})")
        assert n_layers == NUM_LAYERS and D_features_loaded == D_FEATURES
    except Exception as e:
        print(f"LỖI: Không thể tải file database: {e}")
        exit()

    # --- Tải ma trận Mahalanobis ---
    try:
        print(f"Đang tải ma trận Mahalanobis: {mahalanobis_data_file}...")
        mahalanobis_data = torch.load(mahalanobis_data_file)
        # Chuyển ma trận lên GPU
        for i in range(NUM_LAYERS):
            layer_name = f"layer_{i+1}"
            mahalanobis_data['means'][layer_name] = mahalanobis_data['means'][layer_name].to(device)
            mahalanobis_data['whitening_matrices'][layer_name] = mahalanobis_data['whitening_matrices'][layer_name].to(device)
        print(" -> Tải và chuyển ma trận lên GPU thành công!")
    except Exception as e:
        print(f"LỖI: Không thể tải file ma trận: {e}")
        exit()

    # --- "Làm trắng" (Whiten) cho từng lớp theo batch ---
    print(f"\nBắt đầu 'làm trắng' {N_samples} vector (theo batch {BATCH_SIZE})...")

    # Tạo tensor mới để lưu kết quả (đặt trên CPU để tiết kiệm VRAM)
    whitened_db_vectors = torch.empty_like(db_vectors, device='cpu', dtype=torch.float32)

    with torch.no_grad():
        for i in tqdm(range(NUM_LAYERS), desc="Xử lý từng Layer"):
            layer_name = f"layer_{i+1}"
            tensor_index = i

            mean = mahalanobis_data['means'][layer_name]
            W = mahalanobis_data['whitening_matrices'][layer_name]

            # Xử lý theo batch
            for start_idx in range(0, N_samples, BATCH_SIZE):
                end_idx = min(start_idx + BATCH_SIZE, N_samples)

                # [B, 2304]
                X_batch = db_vectors[start_idx:end_idx, tensor_index, :].to(device)

                # 1. Căn giữa (Center)
                X_centered = X_batch - mean # mean [2304] sẽ được broadcast

                # 2. "Làm trắng" (Whiten)
                # [B, 2304] @ [2304, 2304] -> [B, 2304]
                X_whitened = torch.matmul(X_centered, W.T)

                # 3. Lưu kết quả (chuyển về CPU)
                whitened_db_vectors[start_idx:end_idx, tensor_index, :] = X_whitened.cpu()

            # Giải phóng ma trận sau khi xong layer
            del mahalanobis_data['means'][layer_name]
            del mahalanobis_data['whitening_matrices'][layer_name]
            torch.cuda.empty_cache()

    # --- Lưu kết quả ---
    print("\nĐang lưu database đã 'làm trắng'...")
    try:
        torch.save(whitened_db_vectors, output_db_file)
        print(f" -> Đã lưu thành công database đã 'làm trắng' vào:\n    {output_db_file}")
    except Exception as e:
        print(f"LỖI: Không thể lưu file output: {e}")

    print("\n--- BƯỚC 2b ('LÀM TRẮNG' DB) HOÀN TẤT! ---")

--- Bắt đầu Bước 2b: 'Làm trắng' (Whitening) toàn bộ Database Train ---
Sử dụng thiết bị: cpu
File database input: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/combined_train_db_vectors_v2_inner_content.pt
File ma trận input: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/mahalanobis_data_v2_inner_content.pt
File database output: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/combined_train_db_vectors_v2_WHITENED.pt

Đang tải database gốc: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/combined_train_db_vectors_v2_inner_content.pt...
 -> Tải thành công! Shape: (15521, 12, 2304)
Đang tải ma trận Mahalanobis: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/mahalanobis_data_v2_inner_content.pt...
 -> Tải và chuyển ma trận lên GPU thành công!

Bắt đầu 'làm trắng' 15521 vector (theo batch 512)...


Xử lý từng Layer:   0%|          | 0/12 [00:00<?, ?it/s]


Đang lưu database đã 'làm trắng'...
 -> Đã lưu thành công database đã 'làm trắng' vào:
    /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/combined_train_db_vectors_v2_WHITENED.pt

--- BƯỚC 2b ('LÀM TRẮNG' DB) HOÀN TẤT! ---
